<a href="https://colab.research.google.com/github/heromiya/GEE-LCM/blob/master/Jamuna_River_Cross_section_Co_ordinates_automatic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
!pip install fiona

     |████████████████████████████████| 15.4 MB 35 kB/s 


In [ ]:
import fiona
import os
from shapely.geometry import shape,MultiPolygon,Point,LineString,Polygon
from shapely.wkt import loads


CrossSection Path is crossSection of river at certain place &&
predicted is shapefile of river extent predicted by CNN


In [ ]:
root_path = 'gdrive/My Drive/Jamuna River/'

crossSection_path=root_path+'All CrossSection/'
predicted_path=root_path+'Predicted_final/'

crossSection_dataset=os.listdir(crossSection_path)
predicted_dataset=os.listdir(predicted_path)


Sorting Cross Section Line and Predicted shapefile  by Name


In [ ]:
def sortDriveFile(x):
    return(x[0:7])

crossSection_dataset=sorted(crossSection_dataset, key = sortDriveFile)  
predicted_dataset=sorted(predicted_dataset,key=sortDriveFile) 



Selecting only shapefile and addind to list.

In [ ]:
crossSection_dataset_list=[]

for shp in crossSection_dataset:
  if shp.endswith('.shp'):
    crossSection_dataset_list.append(shp)

predicted_dataset_list=[]

for shp in predicted_dataset:
  if shp.endswith('.shp'):
    predicted_dataset_list.append(shp)

In [ ]:
predicted_dataset_list

['2007_01_03.shp']

Converting shapefile to geometry of shapely 

In [ ]:
crossSection_shp=[]
for i in range(len(crossSection_dataset_list)):
  crossSection=fiona.open(crossSection_path+crossSection_dataset_list[i])
  crossSection= next(iter(crossSection))
  crossSection_geom = shape(crossSection['geometry']) 
  crossSection_loads=loads(str(crossSection_geom))
  crossSection_shp.append(crossSection_loads)


predicted_shp=[]

for i in range(len(predicted_dataset_list)):
  predicted=fiona.open(predicted_path+predicted_dataset_list[i])
  predricted=next(iter(predicted))
  predicted_geom = MultiPolygon([shape(predicted['geometry']) for predicted in fiona.open(predicted_path+predicted_dataset_list[i])])
  predicted_loads=loads(str(predicted_geom)) 
  predicted_shp.append(predicted_loads)


In [10]:
for i in range (0, len(predicted_dataset_list)):
  predicted_shp[i]=predicted_shp[i].buffer(0)
  print(predicted_shp[i].is_valid)
  predicted_loads=predicted_shp[i]
  name_of_shapefile=(predicted_dataset_list[i].split('.'))[0]
  print(name_of_shapefile)
  number_of_polygon=len(predicted_loads)
  number_of_crossSection=len(crossSection_dataset_list)
  number_of_shapefile=len(predicted_shp)
  coordinateList=[]
  for i in range(number_of_crossSection):
    emptyList=[]
    coordinateList.append(emptyList)

  # for z in range(number_of_shapefile):

  for i in range(number_of_polygon):

    # eachLine_coordinates=[]
    for j in range(number_of_crossSection):
      
      exterior_intersection = predicted_loads[i].exterior.intersection(crossSection_shp[j])
      # print(intersection.geom_type) 
      if exterior_intersection.is_empty:
        pass   
        
      elif exterior_intersection.geom_type.startswith('Multi')  or exterior_intersection.geom_type == 'GeometryCollection':  

        for shp in exterior_intersection:
              # print(shp.wkt)
              shp=loads(shp.wkt)
              if shp.wkt not in coordinateList[j]:
                coordinateList[j].append(shp.wkt)
                
              else:
                pass
                # print("{} already exist".format(shp.wkt))

              

      intersection = predicted_loads[i].intersection(crossSection_shp[j])
      # print(intersection.geom_type) 
      if intersection.is_empty:
        pass

      elif str(intersection).startswith('MULTILINESTRING'): 
        for shp in intersection:
          shp_loads=loads(shp.wkt)
          for l in range(40):
            
            try:
              x=shp_loads.xy[0][l]
              y=shp_loads.xy[1][l]
              # print(x,y)
              point=Point(x,y)
              # points_xy.append(point.wkt)
              if point.wkt not in coordinateList[j]:  
                # point_data=point
                coordinateList[j].append(point.wkt)
              else:
                pass
                # print("{} already exist".format(point_data.wkt))
              
              # coordinateList_all.append(point.wkt)
            except Exception:
              pass
              
      
      elif str(intersection).startswith('LINESTRING'):
        shp_load=loads(intersection.wkt)
        for k in range(40):
          
          try:
            x=shp_load.xy[0][k]
            y=shp_load.xy[1][k]
            # print(x,y)
            point=Point(x,y)
            if point.wkt not in coordinateList[j]:
              point_data=point  
              coordinateList[j].append(point.wkt)
            else:
              pass
              # print("{} already exist".format(point_data.wkt))
            # points_xy.append(point.wkt)
            # coordinateList_all.append(point.wkt)
          except Exception:
            pass
  def sortingCoordinates(x):
    return(x[8:18])
  for i in range(number_of_crossSection):
    for j in range(len(coordinateList[i])):
      coordinateList[i]=sorted(coordinateList[i], key = sortingCoordinates)
  for i in range(34):
    m=(i/34)*100
    m=round(m)
    print(f'{m}% completed')
    for point in coordinateList[i]:
      def negativeCheck():
        xcordn=(loads(point)).x -0.0001
        ycordn=(loads(point)).y -0.0001
        xy_n =Point(xcordn,ycordn)
        return xy_n.within(predicted_loads)

      def positiveCheck():
        xcordp=(loads(point)).x +0.0001
        ycordp=(loads(point)).y +0.0001
        xy_p =Point(xcordp,ycordp)
        return xy_p.within(predicted_loads)

      if negativeCheck()==True and positiveCheck()==True:
        coordinateList[i].remove(point)
      else:
        pass
  
  import gspread
  from oauth2client.client import GoogleCredentials
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  # create a new spreadsheet with name
  sh = gc.create(name_of_shapefile)
  # Open our new sheet and add some data.
  wks = gc.open(name_of_shapefile).sheet1
  name_cell_lists = wks.range('A2:A35')
  row_head_lists=wks.range("B1:AZ1")

  for i in range(30):
    k=i*2
    l=k+1
    try:
      row_head_lists[k].value="X{}".format(i+1)
      row_head_lists[l].value="Y{}".format(i+1)
    except Exception:
      pass


  for i in range(len(crossSection_dataset_list)):
    x=(crossSection_dataset_list[i].split('.shp'))[0]
    name_cell_lists[i].value=x

  wks.update_cells(name_cell_lists)
  wks.update_cells(row_head_lists)
  
  cell_list_coordinate=[]
  for i in range(len(coordinateList)):
    cell_list_coordinate.append('B{}:AE{}'.format(i+2,i+2))


  for p in range(len(coordinateList)):
    cells=None
    cells=wks.range(cell_list_coordinate[p])
    for i in range(len(coordinateList[p])):
      k=i*2
      l=k+1  
      try:
        cells[k].value=str(loads(coordinateList[p][i]).x)
        cells[l].value=str(loads(coordinateList[p][i]).y)
      except Exception:
        pass
    wks.update_cells(cells)


True
2007_01_03
0% completed
3% completed
6% completed
9% completed
12% completed
15% completed
18% completed
21% completed
24% completed
26% completed
29% completed
32% completed
35% completed
38% completed
41% completed
44% completed
47% completed
50% completed
53% completed
56% completed
59% completed
62% completed
65% completed
68% completed
71% completed
74% completed
76% completed
79% completed
82% completed
85% completed
88% completed
91% completed
94% completed
97% completed
